In [16]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import TrainingArguments, Trainer, GPT2Tokenizer, default_data_collator
from typing import Optional
import logging
from torch.utils.tensorboard import SummaryWriter
from model_hf import GPTConfig, GPTLMHeadModel, MoEUsageLoggingCallback
import numpy as np


In [17]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("training.log", mode='w'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger('MoELogger')


# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# Add special tokens, such as PAD token (if not already present)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})  # Set pad token to eos token



loading file vocab.json from cache at C:\Users\Kosaraju\.cache\huggingface\hub\models--gpt2\snapshots\607a30d783dfa663caf39e06633721c8d4cfcd7e\vocab.json
loading file merges.txt from cache at C:\Users\Kosaraju\.cache\huggingface\hub\models--gpt2\snapshots\607a30d783dfa663caf39e06633721c8d4cfcd7e\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\Kosaraju\.cache\huggingface\hub\models--gpt2\snapshots\607a30d783dfa663caf39e06633721c8d4cfcd7e\tokenizer_config.json
loading file tokenizer.json from cache at C:\Users\Kosaraju\.cache\huggingface\hub\models--gpt2\snapshots\607a30d783dfa663caf39e06633721c8d4cfcd7e\tokenizer.json
loading configuration file config.json from cache at C:\Users\Kosaraju\.cache\huggingface\hub\models--gpt2\snapshots\607a30d783dfa663caf39e06633721c8d4cfcd7e\config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": 

In [5]:
# Prepare the model configuration
block_size = 128

config = GPTConfig(
    block_size=block_size,  # Small block size for debugging
    vocab_size=tokenizer.vocab_size,  # Small vocab size for debugging
    n_layer=4,      # Few layers for faster training
    n_head=4,
    n_embd=256,
    use_moe=True,  # Enable Mixture of Experts
    num_experts=5,
    num_experts_per_tok=2,
    moe_loss=True,
    moe_loss_type = "entropy_regularization",  # Type of load balancing loss "variance_penalty", "entropy_regularization", "diversity_regularization"
    moe_loss_coef = 1e0, 
)

# Initialize the model
model = GPTLMHeadModel(config)

GPT has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Number of parameters: 24.44M


In [7]:
model = GPTLMHeadModel.from_pretrained(pretrained_model_name_or_path="./results", local_files_only=True, device_map='cuda')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Number of parameters: 37.31M


GPTLMHeadModel(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 256)
    (wpe): Embedding(128, 256)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-3): 4 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=256, out_features=768, bias=True)
          (c_proj): Linear(in_features=256, out_features=256, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MoE(
          (experts): ModuleList(
            (0-4): 5 x MLP(
              (c_fc): Linear(in_features=256, out_features=1024, bias=True)
              (gelu): GELU(approximate='none')
              (c_proj): Linear(in_features=1024, out_features=256, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (gate): Linear(in_features=256, out_features=5, bias=False)
        )

In [2]:
import torch
from transformers import GPT2Tokenizer
import logging
from model_hf import GPTLMHeadModel

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("training.log", mode='w'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger('MoELogger')

# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add special tokens, such as PAD token (if not already present)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})  # Set pad token to eos token

# Load the fine-tuned model (from your results folder)
model = GPTLMHeadModel.from_pretrained(pretrained_model_name_or_path="./results", local_files_only=True, device_map='cuda')

# Set the device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Input prompt for text generation
input_text = "To be, or not to be, that is the question:"

# Encode the input text to get input IDs
input_ids = torch.tensor(tokenizer.encode(input_text), device='cuda')[None,...]
# Generate text with the model
generated_ids = model.generate(
    input_ids,
    max_length=100,        # Maximum number of tokens to generate
    temperature=0.7,       # Controls randomness in generation (lower is more deterministic)
    top_k=50,              # Keep only top k tokens with highest probability
    top_p=0.95,            # Nucleus sampling: focus on the top p cumulative probability
    repetition_penalty=1.2,  # Discourage repetition
    do_sample=True,        # Sample from the distribution instead of taking argmax
    num_return_sequences=1  # Number of sequences to generate
)

# Decode the generated text back to readable format
# generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("Generated Text:")
print(generated_text)


Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Number of parameters: 37.31M
Generated Text:
To be, or not to be, that is the question:
ClAnd will upon:Iee you Now good,A and out spirit sleep othersOf too of sovereign,But speak noble.
Second: Henry again earthIs so.
Firstman
 Citizen
ICH:Ay to up the of by own!
INUS
o have person so;For might him do thee
T to his, your,He no state as bear unto fair tongueWhich you, my,'d


In [10]:
# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add special tokens, such as PAD token (if not already present)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})  # Set pad token to eos token

# Load the fine-tuned model
model = GPTLMHeadModel.from_pretrained(pretrained_model_name_or_path="./results", local_files_only=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Input prompt for text generation
input_text = "To be, or not to be, that is the question:"

# Encode the input text to get input IDs
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
if input_ids.size(1) > model.config.block_size:
    input_ids = input_ids[:, :model.config.block_size]
# Generate text with the model
generated_ids = model.generate(
    input_ids,
    max_length=model.config.block_size+1,        # Maximum number of tokens to generate
    temperature=0.9,       # Controls randomness in generation (lower is more deterministic)
    top_k=5000,              # Keep only top k tokens with highest probability
    top_p=0.99,            # Nucleus sampling: focus on the top p cumulative probability
    repetition_penalty=1.2,  # Discourage repetition
    do_sample=True,        # Sample from the distribution instead of taking argmax
    num_return_sequences=1  # Number of sequences to generate
)

# Decode the generated text back to readable format
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Generated Text:")
print(generated_text)


Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Number of parameters: 24.44M
Generated Text:
To be, or not to be, that is the question:
Thirdman
 bear jumps impartialWh her him us him Angelo
 your a and ina for head--As I.
All
Third:The for rough this Cam,. victory
 yourru,.Upon- queen but how.
 high.
Prov:This; honour no,First put another
 his, fellow
 ' to g by himself
 so and is What
ad like
.
aw.What this Can unly You thou,.3 HRY for as first part him again
Prov:It be, manily A judgmentWereI


In [11]:
# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add special tokens, such as PAD token (if not already present)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})  # Set pad token to eos token

# Load the fine-tuned model
model = GPTLMHeadModel.from_pretrained(pretrained_model_name_or_path="./results", local_files_only=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Input prompt for text generation
input_text = "To be, or not to be, that is the question:"

# Encode the input text to get input IDs
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

# Generate text with the model using cache (past_key_values)
generated_ids = model.generate(
    input_ids,
    max_length=model.config.block_size + 50,  # Generate up to 50 tokens beyond block size
    temperature=0.9,       # Controls randomness in generation (lower is more deterministic)
    top_k=50,              # Keep only top k tokens with highest probability
    top_p=0.95,            # Nucleus sampling: focus on the top p cumulative probability
    repetition_penalty=1.2,  # Discourage repetition
    do_sample=True,        # Sample from the distribution instead of taking argmax
    num_return_sequences=1,  # Number of sequences to generate
    use_cache=True         # Enable past_key_values to allow generation beyond block_size
)

# Decode the generated text back to readable format
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Generated Text:")
print(generated_text)


Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Number of parameters: 24.44M


AssertionError: Cannot forward sequence of length 129, block size is only 128

# trainign

In [1]:
import torch
from torch.utils.data import Dataset
from transformers import TrainingArguments, Trainer, GPT2Tokenizer, default_data_collator
from model_hf import GPTConfig, GPTLMHeadModel, MoEUsageLoggingCallback
import numpy as np
import logging
from omegaconf import OmegaConf


In [2]:
# Define the NumpyMemmapDataset class
class NumpyMemmapDataset(Dataset):
    """
    PyTorch Dataset for loading numpy memmapped arrays efficiently.
    """
    def __init__(self, filename="data/mock_train.bin", block_size=1024, device="cpu", iterate='random', 
                 return_labels=False, eval_data=False, eval_samples=1000):
        """
        Args:
        - filename (str): Path to the binary file.
        - block_size (int): Length of the sequence.
        - device (str): Device to move the tensors ('cpu' or 'cuda').
        - iterate (str): 'random' or 'linear'. If 'random', samples random slices from the dataset.
        - return_labels (bool): Whether to return labels (used for GPT-2 LMHeadModel).
        - eval_data (bool): If True, dataset works in evaluation mode and samples a limited number of sequences.
        - eval_samples (int): Number of samples for evaluation mode (if eval_data=True).
        """
        self.data = np.memmap(filename, dtype=np.uint16, mode='r')  # Load binary file with memmap
        self.max_len = len(self.data)
        self.block_size = block_size
        self.iterate = iterate
        self.device = torch.device(device)
        self.return_labels = return_labels
        self.eval_data = eval_data
        self.eval_samples = eval_samples

    def __len__(self):
        # If evaluation mode, return eval_samples, otherwise return full length
        return self.eval_samples if self.eval_data else self.max_len // self.block_size

    def __getitem__(self, index):
        # Handle random sampling if 'iterate' is set to 'random'
        if self.iterate == 'random':
            index = torch.randint(0, self.max_len // self.block_size, (1,)).item()

        idx = index * self.block_size
        x = torch.from_numpy(self.data[idx:idx + self.block_size].astype(np.int64))
        y = torch.from_numpy(self.data[idx + 1:idx + 1 + self.block_size].astype(np.int64))

        if self.device.type == 'cuda':
            # Pin memory for GPU asynchronous transfer
            x = x.pin_memory().to(self.device, non_blocking=True)
            y = y.pin_memory().to(self.device, non_blocking=True)

        # If `return_labels` is true, return both input and target (for GPT training)
        if self.return_labels:
            return {"input_ids": x, "labels": y}
        return x


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("training.log", mode='w'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger('MoELogger')

In [ ]:
# Load config using OmegaConf
config_file = 'config.yaml'  # Path to your config file
cfg = OmegaConf.load(config_file)

# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add special tokens, such as PAD token (if not already present)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})  # Set pad token to eos token


block_size = 128

# Prepare the model configuration
model_config = GPTConfig(**cfg.model)
model = GPTLMHeadModel(model_config)
train_dataset = NumpyMemmapDataset(**cfg.dataset.train)
eval_dataset = NumpyMemmapDataset(**cfg.dataset.eval)
training_args = TrainingArguments(**cfg.train.training_args)
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

# Optional MoE logging callback
moe_logging_callback = MoEUsageLoggingCallback(
    trainer=trainer, logger=logger,**cfg.train.moe_log
)
trainer.add_callback(moe_logging_callback)
# Train the model
trainer.train()

# Save the trained model
trainer.model.save_pretrained(cfg.train.model_save_dir)

c:\Users\Kosaraju\.conda\envs\gpt-moe-env\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
GPT has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify y

Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Using Mixture of Experts (MoE) in MLP
Number of parameters: 24.41M


c:\Users\Kosaraju\.conda\envs\gpt-moe-env\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\Kosaraju\.conda\envs\gpt-moe-env\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend
***** Running training *****
  Num examples = 2,359
  Num Epochs = 85,715
  Instantaneous batch size per device = 30
  Total train batch size (w. parallel, distributed & accumulation) = 300
  Gradient Accumulation steps = 10
  Total optimization steps = 600,000
  Number of trainable parameters = 24,440,320


  0%|          | 0/600000 [00:00<?, ?it/s]

c:\Users\Kosaraju\nanoGPT-1\model_hf.py:141: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  y = F.scaled_dot_product_attention(
2024-10-22 09:22:12,736 - INFO - MoE Layer 1 Usage: Expert 0: 21.09%, Expert 1: 22.10%, Expert 2: 13.25%, Expert 3: 22.28%, Expert 4: 21.28%
2024-10-22 09:22:12,737 - INFO - MoE Layer 2 Usage: Expert 0: 22.25%, Expert 1: 22.40%, Expert 2: 19.73%, Expert 3: 22.45%, Expert 4: 13.17%
2024-10-22 09:22:12,738 - INFO - MoE Layer 3 Usage: Expert 0: 21.33%, Expert 1: 20.86%, Expert 2: 23.40%, Expert 3: 17.23%, Expert 4: 17.18%
2024-10-22 09:22:12,749 - INFO - MoE Layer 4 Usage: Expert 0: 24.09%, Expert 1: 22.09%, Expert 2: 9.04%, Expert 3: 20.45%, Expert 4: 24.33%


...

Logging MoE usage stats...


2024-10-22 09:31:00,516 - INFO - MoE Layer 1 Usage: Expert 0: 21.37%, Expert 1: 18.77%, Expert 2: 18.45%, Expert 3: 20.93%, Expert 4: 20.48%
2024-10-22 09:31:00,517 - INFO - MoE Layer 2 Usage: Expert 0: 20.44%, Expert 1: 19.86%, Expert 2: 19.28%, Expert 3: 21.66%, Expert 4: 18.77%
2024-10-22 09:31:00,517 - INFO - MoE Layer 3 Usage: Expert 0: 16.09%, Expert 1: 22.64%, Expert 2: 22.25%, Expert 3: 19.94%, Expert 4: 19.07%
2024-10-22 09:31:00,519 - INFO - MoE Layer 4 Usage: Expert 0: 22.45%, Expert 1: 21.02%, Expert 2: 18.41%, Expert 3: 25.28%, Expert 4: 12.83%


Logging MoE usage stats...


2024-10-22 09:31:04,314 - INFO - MoE Layer 1 Usage: Expert 0: 21.47%, Expert 1: 18.68%, Expert 2: 18.28%, Expert 3: 21.09%, Expert 4: 20.48%
2024-10-22 09:31:04,315 - INFO - MoE Layer 2 Usage: Expert 0: 20.31%, Expert 1: 19.79%, Expert 2: 19.19%, Expert 3: 21.77%, Expert 4: 18.94%
2024-10-22 09:31:04,316 - INFO - MoE Layer 3 Usage: Expert 0: 16.00%, Expert 1: 22.51%, Expert 2: 22.22%, Expert 3: 20.23%, Expert 4: 19.05%
2024-10-22 09:31:04,317 - INFO - MoE Layer 4 Usage: Expert 0: 22.41%, Expert 1: 21.26%, Expert 2: 18.13%, Expert 3: 25.32%, Expert 4: 12.88%


Logging MoE usage stats...
